1)Если еще не сделано, то: Модифицировать приложение из предыдущего домашнего задания:
- приложение должно собирать вакансии сразу с двух сайтов hh.ru и superjob.ru
- собранные данные должны быть приведены к общей структуре
- разделить зарплату на две составляющие (мин. и макс.) и сохранить в виде int. Если валюта указана другая, привести все к рублям.

2)Реализовать сохранение полученных вакансий в СУБД (на выбор SQLite или MongoDB)
3)Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введенной суммы.
3*)Модифицировать функцию, которая будет добавлять в вашу базу данных только новые вакансии с сайта

In [1]:
import pandas as pd
from pymongo import MongoClient
from pprint import pprint

#### Загрузка датасетов

In [2]:
HH_PATH = 'hh программист с++.csv'
SUPERJOB_PATH = 'superjob программист c#.csv'

In [3]:
hh_df = pd.read_csv(HH_PATH, encoding='utf-8')
hh_df.head(3)

,vacansy,min_salary,max_salary,url,source,employer
0,Программист С++/Qt,70000.0,NaN,https://hh.ru/vacancy/31273030?query=%D0%BF%D1...,hh.ru,Cappasity Inc.
1,Программист С++ (графика),80000.0,NaN,https://hh.ru/vacancy/32543357?query=%D0%BF%D1...,hh.ru,Cappasity Inc.
2,Программист С++,NaN,100000.0,https://hh.ru/vacancy/32417827?query=%D0%BF%D1...,hh.ru,Группа Компаний «АМЕ»


In [4]:
sj_df = pd.read_csv(SUPERJOB_PATH, encoding='utf-8')
sj_df.head(3)

,vacansy,min_salary,max_salary,url,source,employer
0,Инженер-программист / Разработчик C#,300000.0,NaN,https://www.superjob.ru/vakansii/inzhener-prog...,superjob.ru,ФГУ НПП Гамма
1,Инженер-программист C#,NaN,NaN,https://www.superjob.ru/vakansii/inzhener-prog...,superjob.ru,Центральная ППК
2,.NET Developer / Разработчик C#,NaN,NaN,https://www.superjob.ru/vakansii/net-developer...,superjob.ru,Информационные технологии


In [5]:
hh_df.dtypes

vacansy        object
min_salary    float64
max_salary    float64
url            object
source         object
employer       object
dtype: object

In [6]:
sj_df.dtypes

vacansy        object
min_salary    float64
max_salary    float64
url            object
source         object
employer       object
dtype: object

#### Заполним NaN значения нулями

In [7]:
values = {'min_salary': 0, 'max_salary': 0}
hh_df = hh_df.fillna(value=values,);
sj_df = sj_df.fillna(value=values);

In [8]:
hh_df.head(3)

,vacansy,min_salary,max_salary,url,source,employer
0,Программист С++/Qt,70000.0,0.0,https://hh.ru/vacancy/31273030?query=%D0%BF%D1...,hh.ru,Cappasity Inc.
1,Программист С++ (графика),80000.0,0.0,https://hh.ru/vacancy/32543357?query=%D0%BF%D1...,hh.ru,Cappasity Inc.
2,Программист С++,0.0,100000.0,https://hh.ru/vacancy/32417827?query=%D0%BF%D1...,hh.ru,Группа Компаний «АМЕ»


In [9]:
sj_df.head(3)

,vacansy,min_salary,max_salary,url,source,employer
0,Инженер-программист / Разработчик C#,300000.0,0.0,https://www.superjob.ru/vakansii/inzhener-prog...,superjob.ru,ФГУ НПП Гамма
1,Инженер-программист C#,0.0,0.0,https://www.superjob.ru/vakansii/inzhener-prog...,superjob.ru,Центральная ППК
2,.NET Developer / Разработчик C#,0.0,0.0,https://www.superjob.ru/vakansii/net-developer...,superjob.ru,Информационные технологии


#### Подключаемся к БД и создаем записи

In [16]:
client = MongoClient('localhost',27017)
db = client['vacancys']

#### Вставляем данные в БД

In [17]:
data = hh_df.to_dict(orient='records')
hh_db = db.hh_cpp
hh_db.insert_many(data)

In [18]:
hh_db.count()

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  """Entry point for launching an IPython kernel.


165

In [32]:
data = sj_df.to_dict(orient='records')
sj_db = db.sj_csharp
sj_db.insert_many(data)

In [33]:
sj_db.count()

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  """Entry point for launching an IPython kernel.


200

#### Функция поиска и вывода на экран вакансий с суммой, вводимой пользователем

In [30]:
def showVacancys(col_name):
    salary = input('Введите сумму: ')
    for post in col_name.find({'max_salary': {'$gt':int(salary)}}):
        pprint(post)

#### Демонстрация работы функции

In [31]:
showVacancys(hh_db)

Введите сумму: 100000
{'_id': ObjectId('5d7cc873a88bb05903421793'),
 'employer': ' МСК БЛ ГРУПП',
 'max_salary': 150000.0,
 'min_salary': 0.0,
 'source': 'hh.ru',
 'url': 'https://hh.ru/vacancy/33514106?query=%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82%20%D1%81%2B%2B',
 'vacansy': 'Программист С++'}
{'_id': ObjectId('5d7cc873a88bb05903421799'),
 'employer': ' ФОРМ',
 'max_salary': 170000.0,
 'min_salary': 115000.0,
 'source': 'hh.ru',
 'url': 'https://hh.ru/vacancy/33289931?query=%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82%20%D1%81%2B%2B',
 'vacansy': 'Ведущий инженер-программист С++'}
{'_id': ObjectId('5d7cc873a88bb0590342179d'),
 'employer': ' ЮРИОН',
 'max_salary': 110000.0,
 'min_salary': 90000.0,
 'source': 'hh.ru',
 'url': 'https://hh.ru/vacancy/33483197?query=%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82%20%D1%81%2B%2B',
 'vacansy': 'Программист С++'}
{'_id': ObjectId('5d7cc873a88bb0590342179f'),
 'employer': 'ОО

In [34]:
showVacancys(sj_db)

Введите сумму: 100000
{'_id': ObjectId('5d7cc9f5a88bb05903421839'),
 'employer': 'Минерал-Инфо',
 'max_salary': 200000.0,
 'min_salary': 150000.0,
 'source': 'superjob.ru',
 'url': 'https://www.superjob.ru/vakansii/web-programmist-29723693.html',
 'vacansy': 'Web-программист / Web-разработчик'}
{'_id': ObjectId('5d7cc9f5a88bb0590342183d'),
 'employer': 'Кадровый Центр на Сокольнической Площади',
 'max_salary': 250000.0,
 'min_salary': 150000.0,
 'source': 'superjob.ru',
 'url': 'https://www.superjob.ru/vakansii/razrabotchik-c-32357613.html',
 'vacansy': 'Разработчик C#'}
{'_id': ObjectId('5d7cc9f5a88bb0590342183e'),
 'employer': '«Группа ГУТА»',
 'max_salary': 190000.0,
 'min_salary': 150000.0,
 'source': 'superjob.ru',
 'url': 'https://www.superjob.ru/vakansii/full-stack-developer-32491461.html',
 'vacansy': 'Full-Stack developer (Angular, C#, .NET)'}
{'_id': ObjectId('5d7cc9f5a88bb05903421845'),
 'employer': 'НПП Гарант Сервис Университет',
 'max_salary': 140000.0,
 'min_salary': 120